# Agriculture Detection with Keras
This notebook focuses on the TensorFlow/Keras workflow for loading satellite imagery data, batching it, and preparing it for model training.

In [ ]:
# Import Required Libraries
import os
import importlib
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Local imports
import utils.main_utils as main_utils
importlib.reload(main_utils)
from utils.main_utils import check_skillnetwork_extraction, shuffle_data
import utils.keras_ai_utils as keras_ai_utils
importlib.reload(keras_ai_utils)
from utils.keras_ai_utils import (
    set_tf_processing_env,
    create_keras_datasets,
    display_keras_batch,
    configure_keras_for_performance,
    keras_custom_data_generator,
    display_custom_keras_batch,
    create_generators,
    build_keras_model,
    display_keras_history,
    evaluate_keras_model
)

<!-- ## Download and Extract Data
Download the satellite images dataset and extract it to the `data/` directory if it is not already available. -->

In [ ]:
# Download and extract data
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/4Z1fwRR295-1O3PMQBH6Dg/images-dataSAT.tar'
extract_dir = './data/tf_data/'
model_dir = './models/'
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
dataset_path = os.path.join(extract_dir, 'images_dataSAT')
await check_skillnetwork_extraction(extract_dir, url)

## Label and Shuffle Data
Aggregate image paths and labels for agricultural and non-agricultural classes, then shuffle for downstream batching.

In [ ]:
# Label and shuffle data
base_dir = './data/tf_data/images_dataSAT/'
dir_non_agri_name = os.path.join(base_dir, 'class_0_non_agri')
dir_agri_name = os.path.join(base_dir, 'class_1_agri')

all_image_paths, all_labels = shuffle_data(dir_non_agri_name, dir_agri_name)

## Custom Data Generator vs Keras Dataset

In [ ]:
# Custom generator for custom batching
batch_size = 8
data_generator = keras_custom_data_generator(
    image_paths=all_image_paths,
    labels=all_labels,
    batch_size=batch_size,
)
display_custom_keras_batch(data_generator, batch_size, title='Custom Generator')

# Create and inspect Keras datasets
img_size = (64, 64)
train_ds, val_ds = create_keras_datasets(
    base_dir=base_dir,
    img_size=img_size,
    batch_size=batch_size,
)
display_keras_batch(batch_size, train_ds, title='Keras Dataset')

## Configure Keras Pipelines for Performance
Apply caching, shuffling, and prefetching strategies to optimize input pipelines.

In [ ]:
# Optimize datasets for performant training
train_ds, val_ds = configure_keras_for_performance(train_ds, val_ds)

## Setup Tensorflow Processing Environment

In [ ]:
device, fnames = set_tf_processing_env(dataset_path)

## Define Hyperparameters

In [ ]:
# Model Architecture Parameters
img_w, img_h = 64, 64                  # Image width and height
n_channels = 3                         # Number of image channels (e.g., 3 for RGB)
hidden_activation = 'relu'             # Activation function for hidden layers
output_activation = 'sigmoid'          # Activation function for the output layer
padding = 'same'                       # Padding strategy for convolutional layers
strides = (1, 1)                       # Strides for convolutional layers
kernel_size = (3, 3)                   # Kernel size for convolutional layers
kernel_initializer = HeUniform()       # Initializer for the kernel weights
kernel_regularizer = l2(0.001)         # Regularizer for the kernel weights
dropout = .2                           # Dropout rate for regularization
filter_base = 32                       # Base number of filters for convolutional layers
unit_base = 128                        # Base number of units for dense layers
output_units = 1                       # Number of units in the output layer
conv_block_num = 3                     # Number of convolutional blocks
dense_block_num = 1                    # Number of dense blocks
pool_size = (2, 2)                     # Pool size for MaxPooling layers
pool_strides = (2, 2)                  # Strides for MaxPooling layers

# Training Hyperparameters
lr = .001                              # Learning rate for the optimizer
n_epochs = 10                          # Number of times to iterate over the entire dataset
batch_size = 128                       # Number of samples per gradient update
steps_per_epoch = None                 # Number of steps (batches) to take for each epoch
validation_steps = None                # Number of steps (batches) to take for validation
loss = 'binary_crossentropy'           # Loss function for the model
optimizer = Adam(learning_rate=lr)     # Optimizer for training the model
metrics = ['accuracy']                 # Metrics to monitor during training

# Data Augmentation Parameters
rescale = 1./255                       # Rescaling factor for pixel values
rotation_range = 20                    # Degree range for random rotations
width_shift_range = 0.1                # Fraction of total width for random horizontal shifts
height_shift_range = 0.1               # Fraction of total height for random vertical shifts
shear_range = 0.1                      # Shear intensity (shear angle in counter-clockwise direction)
zoom_range = 0.075                     # Range for random zoom
horizontal_flip = True                 # Randomly flip inputs horizontally
fill_mode = 'nearest'                  # Strategy for filling in newly created pixels
validation_split = 0.2                 # Fraction of data to reserve for validation

# Configuration Parameters
model_name = os.path.join(             # Name of the file to save the model
    model_dir, 
    'keras_model.model.keras'
) 
class_mode = 'binary'                  # Type of classification problem
target_size = (img_w, img_h)           # Dimensions to which all images will be resized
tqdm_verbose = 1                       # Verbosity mode for TQDM progress bar
verbose = 0                            # Verbosity mode for model training

## Create Data Generators for Training and Validation

In [ ]:
train_generator, validation_generator = create_generators(
    dataset_path,
    target_size,
    batch_size,
    class_mode,
    rescale,
    rotation_range,
    width_shift_range,
    height_shift_range,
    shear_range,
    zoom_range,
    horizontal_flip,
    fill_mode,
    validation_split
)

## Build and Train Model

In [ ]:
model, fit = build_keras_model(
    train_generator,
    validation_generator,
    optimizer,
    kernel_initializer, 
    filter_base=filter_base, 
    unit_base=unit_base, 
    output_units=output_units, 
    conv_block_num=conv_block_num,
    dense_block_num=dense_block_num,
    hidden_activation=hidden_activation, 
    padding=padding, 
    strides=strides, 
    kernel_size=kernel_size, 
    img_w=img_w, 
    img_h=img_h, 
    n_channels=n_channels, 
    dropout=dropout, 
    output_activation=output_activation,
    loss=loss,
    metrics=metrics,
    model_name=model_name,
    device=device,
    batch_size=batch_size,
    lr=lr,
    n_epochs=n_epochs,
    steps_per_epoch=steps_per_epoch,
    tqdm_verbose=tqdm_verbose,
    verbose=verbose,
    validation_steps=validation_steps,
    pool_size=pool_size,
    pool_strides=pool_strides
)

## Display Results

In [ ]:
display_keras_history(validation_generator, model, fit)

## Display Metrics

In [ ]:
evaluate_keras_model(model, validation_generator, device, model_name)